In [1]:
import os

In [2]:
%pwd

'c:\\Projects\\NLP\\NLP End to End Krish\\TextSummarizationNLP\\research'

In [3]:
os.chdir("..")

In [4]:
%pwd

'c:\\Projects\\NLP\\NLP End to End Krish\\TextSummarizationNLP'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [15]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

PARAM_FILE_PATH

WindowsPath('params.yaml')

In [19]:
class ConfigurationManager:
    def __init__(self, config_path = CONFIG_FILE_PATH, param_path = PARAM_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml( param_path)
        create_directories([self.config.artifacts_root])

    def getDataIngestionConfig(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        return DataIngestionConfig(
            root_dir=config.root_dir, 
            source_URL=config.source_URL, 
            local_data_file=config.local_data_file, 
            unzip_dir=config.unzip_dir
        )
    
    

In [9]:
import urllib.request as request
import zipfile
from TextSummarizer.logging import logger
from TextSummarizer.utils.common import get_size

In [20]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: {headers}")
        else:
            logger.info(f"File already exists with file size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip(self):
        unzip_dir = self.config.unzip_dir
        os.makedirs(unzip_dir, exist_ok=True)
        with zipfile.ZipFile(file=self.config.local_data_file, mode='r') as zipRef:
            zipRef.extractall(unzip_dir)
        logger.info("Extraction of file complete")

            

In [21]:

try:
    config_manager = ConfigurationManager()
    data_ingestion = DataIngestion(config=config_manager.getDataIngestionConfig())
    data_ingestion.download_file()
    data_ingestion.extract_zip()
except Exception as e:
    raise e


[2024-09-22 06:16:58,804 : INFO : common : YAML File : config\config.yaml loaded successfully]
[2024-09-22 06:16:58,806 : INFO : common : YAML File : params.yaml loaded successfully]
[2024-09-22 06:16:58,808 : INFO : common : Created directory : artifacts]
[2024-09-22 06:16:58,808 : INFO : common : Created directory : artifacts/data_ingestion]
[2024-09-22 06:17:01,192 : INFO : 1304962590 : artifacts/data_ingestion/data.zip downloaded with following info: Connection: close
Content-Length: 4130809
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "efd2820c306e31c8208cefe9e9f37ed63d04a45d66b220dbd54de38c009566ea"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 83A2:3C19B7:1065760:1207F98:66EFEE9D
Accept-Ranges: bytes
Date: Sun, 22 Sep 2024 10:17:03 GMT
Via: 1.1 varnish
X-Served-By: cache-ewr-kewr